<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/3f/HubSpot_Logo.svg/220px-HubSpot_Logo.svg.png" alt="drawing" width="200" align='left'/>

# Hubspot - Update followers from linkedinbio
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Hubspot/Hubspot_update_followers_from_linkedinbio.ipynb" target="_parent"><img src="https://img.shields.io/badge/-Open%20in%20Naas-success?labelColor=000000&logo="/></a>

## Input

### Import library

In [ ]:
from naas_drivers import hubspot, linkedin
import naas
import pandas as pd

### Enter Hubspot api key

In [ ]:
auth_token = "YOUR_HUBSPOT_API_KEY"

### Get your cookies
<a href='https://www.notion.so/LinkedIn-driver-Get-your-cookies-d20a8e7e508e42af8a5b52e33f3dba75'>How to get your cookies ?</a>

In [ ]:
LI_AT = 'YOUR_COOKIE_LI_AT'  # EXAMPLE AQFAzQN_PLPR4wAAAXc-FCKmgiMit5FLdY1af3-2
JSESSIONID = 'YOUR_COOKIE_JSESSIONID'  # EXAMPLE ajax:8379907400220387585

### Connect to hubspot

In [ ]:
hs = hubspot.connect(auth_token)

### Schedule your notebook everyday

In [ ]:
naas.scheduler.add(cron="15 6 * * *")

### Get all contacts in Hubspot

In [ ]:
properties_list = [
    "hs_object_id",
    "firstname",
    "lastname",
    "linkedinbio",
    "linkedinconnections",
]
hubspot_contacts = hs.contacts.get_all(properties_list).fillna("Not Defined")
hubspot_contacts

# Model

### Filter to get linkedinconnections "Not Defined" and "linkedinbio" defined

In [ ]:
df_to_update = hubspot_contacts.copy()

# Filter on "Not defined"
df_to_update = df_to_update[(df_to_update.linkedinbio != "Not Defined") &
                            (df_to_update.linkedinconnections == "Not Defined")]

# Limit to last 50 contacts
df_to_update = df_to_update.sort_values(by="createdate", ascending=False)[:50].reset_index(drop=True)

df_to_update

### Get followers from Linkedin

In [ ]:
for _, row in df_to_update.iterrows():
    linkedinbio = row.linkedinbio
    
    # Get followers
    df = linkedin.connect(LI_AT, JSESSIONID).profile.get_network(linkedinbio)
    linkedinconnections = df.loc[0, "FOLLOWERS_COUNT"]
        
    # Get linkedinbio
    df_to_update.loc[_, "linkedinconnections"] = linkedinconnections
    
df_to_update

# Output

### Update followers in Hubspot

In [ ]:
for _, row in df_to_update.iterrows():
    # Init data
    data = {}
    
    # Get data
    hs_object_id = row.hs_object_id
    linkedinconnections = row.linkedinconnections

    # Update LK Bio
    if linkedinconnections != None:
        data = {"properties": {"linkedinconnections": linkedinconnections}}
    hs.contacts.patch(hs_object_id, data)